In [13]:
import pandas as pd
import numpy as np
import os
from shutil import copyfile
import shutil
import re
from email_validator import validate_email, EmailNotValidError
import datetime
from datetime import date
import mysql.connector

In [14]:
WORKFILES = "home/vinkOS/archivosVisitas/"

In [15]:
columns = ['email', 'jk', 'Badmail', 'Baja', 'Fecha envio', 'Fecha open', 'Opens',
           'Opens virales', 'Fecha click', 'Clicks', 'Clicks virales', 'Links',
           'IPs', 'Navegadores', 'Plataformas']

dtype = ['str', 'str', 'str', 'str', 'str', 'str', 'int64', 'int64', 'str', 
         'int64', 'int64', 'str', 'str', 'str', 'str']    

In [16]:
def create_dataframe(columns, dtype):
    
    cdt = {i[0]: i[1] for i in zip(columns, dtype)}  # make column type dict
    df = pd.DataFrame(columns = list(cdt))           # create empty dataframe
    df = df.astype(cdt)                              # set types 
    
    return df

In [17]:
def check_layout(filename, df1, columns, dtype):
   
    df2 = create_dataframe(columns, dtype)
    
    if not (len(df1.columns) == len(df2.columns)):
        print(f"Number of DataFrame column names are different, filename: {filename}")
        return False
    else:
        if not (df1.columns == df2.columns).all() :
            print(f"DataFrame column names are different, filename: {filename}")
            return False
        else:
            if any(df1.dtypes != df2.dtypes):
                print(f"Data Types are different, filename: {filename}")
                return False
    
    return True    

In [18]:
def load_files(folder):
    
    files = [f for f in os.listdir(folder) if re.match(r'report_+[0-9]+.*\.txt', f)]
    
    dataframes_list = []
    
    for f in files:
        
        dataframes_list.append((f, pd.read_csv((folder+f), delimiter = ',')))
        
    return dataframes_list        

In [19]:
# Load files
dataframes_list = load_files(WORKFILES)

In [20]:
# Check Layout
for filename, df in dataframes_list:
    if check_layout(filename, df, columns, dtype):
        print(f"File {filename} is OK")
    else:
        print(f"File {filename} is NOT OK")
    print("")

Data Types are different, filename: report_7.txt
File report_7.txt is NOT OK

DataFrame column names are different, filename: report_8.txt
File report_8.txt is NOT OK

DataFrame column names are different, filename: report_9.txt
File report_9.txt is NOT OK



In [21]:
def check_email(string):
    
    try:
    
        valid = validate_email(string)
        return True
    
    except:
    
        return False
    
def check_date(string):
    
    try:
        
        datetime.datetime.strptime(string, "%d/%m/%Y %H:%M")
        return True
        
    except:
        
        return False
    
def validate_information(df):
    
    df_validated = pd.DataFrame(columns = df.columns)
    df_errors = pd.DataFrame(columns = df.columns)
   
    for index, row in df.iterrows():
        
        if ( check_email(row['email']) and check_date(row['Fecha envio']) and
             check_date(row['Fecha open']) and check_date(row['Fecha click']) ):
            
            df_validated = df_validated.append(row)
            
        else:
                        
            df_errors = df_errors.append(row)
    
    return ( (df_validated, df_errors) )   

In [22]:
df_validated_acum = create_dataframe(columns, dtype)
df_errors_acum = create_dataframe(columns, dtype)

for filename, df in dataframes_list:
    
    print(f"Validating: {filename} ...")
    df_validated, df_errors = validate_information(df)
    print(f"Rows validated GOOD: {len(df_validated)}")
    print(f"Rows with ERRORS: {len(df_errors)}")
    
    df_validated_acum = df_validated_acum.append(df_validated)
    df_errors_acum = df_errors_acum.append(df_errors)
    
    print(f"Finished: {filename}\n")

Validating: report_7.txt ...
Rows validated GOOD: 23
Rows with ERRORS: 480
Finished: report_7.txt

Validating: report_8.txt ...
Rows validated GOOD: 10
Rows with ERRORS: 493
Finished: report_8.txt

Validating: report_9.txt ...
Rows validated GOOD: 11
Rows with ERRORS: 984
Finished: report_9.txt



In [23]:
df_validated_acum.fillna('', inplace=True)
df_errors_acum.fillna('', inplace=True)

In [24]:
print(f"Total Rows validated GOOD: {len(df_validated_acum)}")
print(f"Total Rows with ERRORS: {len(df_errors_acum)}")

Total Rows validated GOOD: 44
Total Rows with ERRORS: 1957


# SQL

In [25]:
from mysql.connector import connect, Error

In [26]:
DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASSWORD = 'Eduardo_69'
DB_NAME = 'Vinkos'

In [27]:
import mysql.connector
from mysql.connector import errorcode

config = {
    'user': DB_USER,
    'password': DB_PASSWORD,
    'host': DB_HOST,
    'database': DB_NAME
}

db = mysql.connector.connect(**config)
cursor = db.cursor()

In [28]:
TABLES = {}

TABLES['visitante'] = (
    "CREATE TABLE visitante ("
    " email              VARCHAR(254) NOT NULL,"
    " fechaprimeravisita DATE NOT NULL,"
    " fechaultimavisita  DATE NOT NULL,"
    " visitastotales     INT(10),"
    " visitasanioactual  INT(10),"
    " visitasmesactual   INT(10),"
    " PRIMARY KEY (email) ) "
)

TABLES['estadistica'] = (
    "CREATE TABLE estadistica ("
    " email          VARCHAR(254) NOT NULL,"
    " jyv            VARCHAR(250),"
    " badmail        VARCHAR(254),"
    " baja           VARCHAR(250),"
    " fecha_envio    DATE,"
    " fecha_open     DATE,"
    " opens          INT(10),"
    " opens_virales  INT(10),"
    " fecha_click    VARCHAR(10),"
    " clicks         INT(10),"
    " clicks_virales INT(10),"
    " links          VARCHAR(250),"
    " ips            VARCHAR(250),"
    " navegadores    VARCHAR(250),"
    " plataformas    VARCHAR(250) )")
    
"""    
    " PRIMARY KEY (email) )"
)
"""

TABLES['errores'] = (
    "CREATE TABLE errores ("
    " email          VARCHAR(254) NOT NULL,"
    " jyv            VARCHAR(250),"
    " badmail        VARCHAR(254),"
    " baja           VARCHAR(250),"
    " fecha_envio    VARCHAR(250),"
    " fecha_open     VARCHAR(250),"
    " opens          INT(10),"
    " opens_virales  INT(10),"
    " fecha_click    VARCHAR(250),"
    " clicks         INT(10),"
    " clicks_virales INT(10),"
    " links          VARCHAR(250),"
    " ips            VARCHAR(250),"
    " navegadores    VARCHAR(250),"
    " plataformas    VARCHAR(250) )")

In [29]:
def create_database():
    cursor.execute(
        "CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    print("Database {} created!".format(DB_NAME))

In [30]:
def create_tables():
    
    cursor.execute("USE {}".format(DB_NAME))
    for table_name in TABLES:
        table_description = TABLES[table_name]
        try:
            print("Creating table ({}) ".format(table_name), end="")
            cursor.execute(table_description)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("Already Exists")
            else:
                print(err.msg)

In [31]:
create_database()

Database Vinkos created!


In [32]:
create_tables()

Creating table (visitante) Already Exists
Creating table (estadistica) Already Exists
Creating table (errores) Already Exists


In [33]:
def add_update_visitante(email, fechaprimeravisita, fechaultimavisita, 
                         visitastotales, visitasanioactual, visitasmesactual):
    
    
    d = datetime.datetime.strptime(fechaprimeravisita, "%d/%m/%Y %H:%M")
    fechaprimeravisita = d.strftime('%Y%m%d')
    
    d = datetime.datetime.strptime(fechaultimavisita, "%d/%m/%Y %H:%M")
    fechaultimavisita = d.strftime('%Y%m%d')
    
    
    sql = ("INSERT INTO visitante (email, fechaprimeravisita, fechaultimavisita," 
           "                       visitastotales, visitasanioactual, visitasmesactual) "
           "               VALUES (%s, %s, %s, %s, %s, %s) "
           "   ON DUPLICATE KEY UPDATE fechaultimavisita = %s, "
           "                           visitastotales = visitastotales + 1, "
           "                           visitasanioactual = visitasanioactual + 1, "
           "                           visitasmesactual = visitasmesactual + 1 ")
    cursor.execute(sql, (email, fechaprimeravisita, fechaultimavisita, 
                         visitastotales, visitasanioactual, visitasmesactual,
                         fechaultimavisita))
    db.commit()

In [34]:
n = 0

print(f"Adding/Updating 'visitantes' table ...")
    
print("Updating row number: ")
for index, row in df_validated_acum.iterrows():
        
    add_update_visitante(row['email'], row['Fecha envio'], row['Fecha open'], 1, 1, 1)
    n += 1
    print(n, ", ", end='')
        
print(f"\nAdding/Updating 'visitantes' table is FINISHED\n")
        
        

Adding/Updating 'visitantes' table ...
Updating row number: 
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 , 36 , 37 , 38 , 39 , 40 , 41 , 42 , 43 , 44 , 
Adding/Updating 'visitantes' table is FINISHED



In [35]:
def add_estadistica(email, jk, Badmail, Baja, Fecha_envio, Fecha_open, Opens, Opens_virales, 
                    Fecha_click, Clicks, Clicks_virales, Links, IPs, Navegadores, 
                    Plataformas):
    
    d = datetime.datetime.strptime(Fecha_envio, "%d/%m/%Y %H:%M")
    Fecha_envio = d.strftime('%Y%m%d')
    
    d = datetime.datetime.strptime(Fecha_open, "%d/%m/%Y %H:%M")
    Fecha_open = d.strftime('%Y%m%d')
    
    d = datetime.datetime.strptime(Fecha_click, "%d/%m/%Y %H:%M")
    Fecha_click = d.strftime('%Y%m%d')
    
    sql = ("INSERT INTO estadistica (email, jyv, badmail, baja, fecha_envio, fecha_open, opens, "
           "                         opens_virales, fecha_click, clicks, clicks_virales, links, "
           "                         ips, navegadores, plataformas) "
           "               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ")
    
    cursor.execute(sql, (email, jk, Badmail, Baja, Fecha_envio, Fecha_open, Opens, Opens_virales,
                           Fecha_click, Clicks, Clicks_virales, Links, IPs, Navegadores, 
                           Plataformas))
    db.commit()

In [36]:
n = 0

print(f"Adding 'estadistica' table ...")
    
print("Adding row number: ")
for index, row in df_validated_acum.iterrows():
        
    add_estadistica(row['email'], row['jk'], row['Badmail'], row['Baja'], row['Fecha envio'], 
                    row['Fecha open'], row['Opens'], row['Opens virales'],
                    row['Fecha click'], row['Clicks'], row['Clicks virales'],
                    row['Links'], row['IPs'], row['Navegadores'], row['Plataformas'])
    
    n += 1
    print(n, ", ", end='')
        
print(f"\nAdding 'estadistica' table is FINISHED\n")
        

Adding 'estadistica' table ...
Adding row number: 
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 , 36 , 37 , 38 , 39 , 40 , 41 , 42 , 43 , 44 , 
Adding 'estadistica' table is FINISHED



In [37]:
def add_errores(email, jk, Badmail, Baja, Fecha_envio, Fecha_open, Opens, Opens_virales, 
                Fecha_click, Clicks, Clicks_virales, Links, IPs, Navegadores, 
                Plataformas):
    
    sql = ("INSERT INTO errores (email, jyv, badmail, baja, fecha_envio, fecha_open, opens, "
           "                     opens_virales, fecha_click, clicks, clicks_virales, links, "
           "                     ips, navegadores, plataformas) "
           "               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ")
    
    cursor.execute(sql, (email, jk, Badmail, Baja, Fecha_envio, Fecha_open, Opens, Opens_virales,
                         Fecha_click, Clicks, Clicks_virales, Links, IPs, Navegadores, 
                         Plataformas))
    db.commit()

In [38]:
n = 0

print(f"Adding 'errores' table ...")
    
print("Adding row number: ")
#for index, row in df_validated_acum.iterrows():
for index, row in df_errors_acum.iterrows():
        
    add_errores(row['email'], row['jk'], row['Badmail'], row['Baja'], row['Fecha envio'], 
                row['Fecha open'], row['Opens'], row['Opens virales'],
                row['Fecha click'], row['Clicks'], row['Clicks virales'],
                row['Links'], row['IPs'], row['Navegadores'], row['Plataformas'])
    
    n += 1
    print(n, ", ", end='')
        
print(f"\nAdding 'errores' table is FINISHED\n")
        

Adding 'errores' table ...
Adding row number: 
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 , 36 , 37 , 38 , 39 , 40 , 41 , 42 , 43 , 44 , 
Adding 'errores' table is FINISHED



# Remove Files

In [39]:
files = [WORKFILES+dataframes_list[idx][0] for idx in range(0, len(dataframes_list))]

In [40]:
files

['home/vinkOS/archivosVisitas/report_7.txt',
 'home/vinkOS/archivosVisitas/report_8.txt',
 'home/vinkOS/archivosVisitas/report_9.txt']

In [41]:
for f in files:
    if os.path.exists(f):
        os.remove(f)
        print(f"File {f} removed")
    else:
        print(f"The file: {f} does not exist")

File home/vinkOS/archivosVisitas/report_7.txt removed
File home/vinkOS/archivosVisitas/report_8.txt removed
File home/vinkOS/archivosVisitas/report_9.txt removed


In [42]:
"""
def copy_files():
    copyfile('home/vinkOS/archivosVisitas/backup/report_7.txt', files[0])
    copyfile('home/vinkOS/archivosVisitas/backup/report_8.txt', files[1])
    copyfile('home/vinkOS/archivosVisitas/backup/report_9.txt', files[2])
"""

In [43]:
#copy_files()

# Backup files

In [44]:
import os
import datetime
from zipfile import ZipFile

In [45]:
BACKUP_DIR_NAME = WORKFILES + "backups"
FILE_PREFIX = "backup_"
FILE_SUFFIX_DATE_FORMAT = "%Y%m%d%H%M%S"
USERNAME = DB_USER
DBNAME = DB_NAME

In [46]:
# Change in my.ini file in order to save backups in this folder

"""
# Secure File Priv.
secure-file-priv=""

# Path to the database root
datadir=/home/etl/visitas/bckp

"""

'\n# Secure File Priv.\nsecure-file-priv=""\n\n# Path to the database root\ndatadir=/home/etl/visitas/bckp\n\n'

# ZIP files

In [47]:
# Obtain datadir value
sql = ("SELECT @@datadir")
cursor.execute(sql)
datadir = cursor.fetchall()
datadir = datadir[0][0]

In [48]:
# get today's date and time
timestamp = datetime.datetime.now().strftime(FILE_SUFFIX_DATE_FORMAT)
backup_filename1 = FILE_PREFIX + "visitante_" + timestamp + ".sql"
backup_filename2 = FILE_PREFIX + "estadistica_" + timestamp + ".sql"
backup_filename3 = FILE_PREFIX + "errores_" + timestamp + ".sql"

In [49]:
backup = ("SELECT * INTO OUTFILE '" + backup_filename1 + "' FROM visitante;"
          "SELECT * INTO OUTFILE '" + backup_filename2 + "' FROM estadistica;"
          "SELECT * INTO OUTFILE '" + backup_filename3 + "' FROM errores;")

In [50]:
cursor.execute(backup)

In [51]:
print(f"File: {backup_filename1} , Ready")
print(f"File: {backup_filename2} , Ready")
print(f"File: {backup_filename3} , Ready")

File: backup_visitante_20220106225837.sql , Ready
File: backup_estadistica_20220106225837.sql , Ready
File: backup_errores_20220106225837.sql , Ready


In [56]:
db.close()

In [52]:
zip_filename1 = FILE_PREFIX + "visitante_" + timestamp + ".zip"
zip_filename2 = FILE_PREFIX + "estadistica_" + timestamp + ".zip"
zip_filename3 = FILE_PREFIX + "errores_" + timestamp + ".zip"

In [53]:
shutil.make_archive(datadir + "vinkos\\"+backup_filename1, 'zip', datadir+"vinkos\\")
print(f"File: {backup_filename1}.zip , Ready")

File: backup_visitante_20220106225837.sql.zip , Ready


In [54]:
shutil.make_archive(datadir + "vinkos\\"+backup_filename2, 'zip', datadir+"vinkos\\")
print(f"File: {backup_filename2}.zip , Ready")

File: backup_estadistica_20220106225837.sql.zip , Ready


In [55]:
shutil.make_archive(datadir + "vinkos\\"+backup_filename3, 'zip', datadir+"vinkos\\")
print(f"File: {backup_filename3}.zip , Ready")

File: backup_errores_20220106225837.sql.zip , Ready


# Bitácora de control de carga de los archivos

In [59]:
file = open("bitacora.txt", "w")
file.write("-----------------------------------------" + os.linesep)
file.write(f"Fecha: {date.today()}" + os.linesep)
for f in files:
    file.write(f"{f} loaded" + os.linesep)
file.write(f"Total Rows validated GOOD: {len(df_validated_acum)}" + os.linesep)
file.write(f"Total Rows with ERRORS: {len(df_errors_acum)}" + os.linesep)
file.write("-----------------------------------------" + os.linesep)
file.close()